In [448]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score

In [463]:
data = pd.read_csv("C:\Users\Niku\Documents\ML Project\data10000.csv")
print data.shape
data['Date'] = pd.to_datetime(data['Date'])
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['DateOnly'] = data['Date'].dt.day
print data.shape

(10000, 13)
(10000, 16)


In [488]:
X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.15
#X = data[['DayOfWeek', 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly']] #Acc = 0.16
#X = data[['DayOfWeek',  'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.155
#X = data[[ 'PdDistrict', 'X', 'Y', 'Year', 'Month', 'DateOnly', 'Time']] #Acc = 0.17 with test size 0.05
print X.shape
X = X.values
y = data['Category']
y = y.values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state=1)
print X_train.shape
print X_test.shape
print y_test.shape

(10000, 8)
(5000L, 8L)
(5000L, 8L)
(5000L,)


In [489]:
def Prob(y_vector):
    total = len(y_vector)
    r_dict = {}
    cnt = Counter(y_vector)
    for key, value in cnt.items():
        r_dict[key] = float(value)/total
    return r_dict
    
dic = {}  
#ClassifyingByLabels 
def ClassifyByLabels(X_matrix, y_vector):
    
    [rows, columns] = np.shape(X_matrix)
    total = rows
    labels = np.unique(y_vector)
    for label in labels:
        dic[label] = defaultdict(list)
    for label in labels:
        row_indices = np.where(y_vector == label)[0] 
        X_current = X_train[row_indices, :]
        for i in range(0,columns):
            dic[label][i] += list(X_current[:,i])
    for label in labels:
        for col in range(0, columns):
            dic[label][col] = Prob(dic[label][col])
            #print dic[label][col]
            
ClassifyByLabels(X_train, y_train)   
crime_dict = Prob(y_train)
crime_dict

{'ARSON': 0.0028,
 'ASSAULT': 0.0968,
 'BAD CHECKS': 0.0002,
 'BRIBERY': 0.0004,
 'BURGLARY': 0.0434,
 'DISORDERLY CONDUCT': 0.0032,
 'DRIVING UNDER THE INFLUENCE': 0.0036,
 'DRUG/NARCOTIC': 0.0324,
 'DRUNKENNESS': 0.0052,
 'EMBEZZLEMENT': 0.0008,
 'EXTORTION': 0.0004,
 'FAMILY OFFENSES': 0.0008,
 'FORGERY/COUNTERFEITING': 0.0038,
 'FRAUD': 0.0274,
 'GAMBLING': 0.0008,
 'KIDNAPPING': 0.0028,
 'LARCENY/THEFT': 0.1642,
 'LIQUOR LAWS': 0.001,
 'MISSING PERSON': 0.0304,
 'NON-CRIMINAL': 0.1356,
 'OTHER OFFENSES': 0.1604,
 'PROSTITUTION': 0.0032,
 'RECOVERED VEHICLE': 0.0072,
 'ROBBERY': 0.0288,
 'RUNAWAY': 0.0008,
 'SECONDARY CODES': 0.0192,
 'SEX OFFENSES, FORCIBLE': 0.0064,
 'SEX OFFENSES, NON FORCIBLE': 0.0004,
 'STOLEN PROPERTY': 0.009,
 'SUICIDE': 0.0002,
 'SUSPICIOUS OCC': 0.0372,
 'TRESPASS': 0.0076,
 'VANDALISM': 0.039,
 'VEHICLE THEFT': 0.0506,
 'WARRANTS': 0.0598,
 'WEAPON LAWS': 0.0142}

In [490]:
#Output maximum probability category
def maximum(dict1):
    values = dict1.values()
    keys = dict1.keys()
    max_index = values.index(max(values))
    max_key = keys[max_index]
    return max_key

#Testing on X_test

def Prediction(elem):
    predict_dict = {}
    for label in labels:
        prior = crime_dict[label]
        likelihood = 1.0
        for i in range(0, len(elem)):
            if elem[i] in dic[label][i].keys():
                likelihood *= dic[label][i][elem[i]]
            else:
                likelihood *= 0.0
        predict_dict[label] = likelihood
    return maximum(predict_dict)   
        

def classify(X_matrix):
        predicted_y = []
        no_rows, no_cols = np.shape(X_matrix)
        for i in range(0,no_rows):
            elem = X_matrix[i,:]
            prediction = Prediction(elem)
            predicted_y.append(prediction)
        return predicted_y

classify(X_test)
predicted_y = classify(X_test)
y_labels = np.unique(y_test)
tt = np.array(predicted_y)
print type(tt)
for y_label in y_labels:
    f1 = f1_score(tt, y_test, y_label, average="weighted")
    print("F1-score on the test-set for class %s is: %s" % (y_label, f1))

<type 'numpy.ndarray'>
F1-score on the test-set for class ARSON is: 0.133041416187
F1-score on the test-set for class ASSAULT is: 0.125517222984
F1-score on the test-set for class BAD CHECKS is: 0.160594333514
F1-score on the test-set for class BRIBERY is: 0.125517222984
F1-score on the test-set for class BURGLARY is: 0.166315342492
F1-score on the test-set for class DISORDERLY CONDUCT is: 0.0536366774134
F1-score on the test-set for class DRIVING UNDER THE INFLUENCE is: 0.078765695458
F1-score on the test-set for class DRUG/NARCOTIC is: 0.1561329958
F1-score on the test-set for class DRUNKENNESS is: 0.160594333514
F1-score on the test-set for class EMBEZZLEMENT is: 0.158712476994
F1-score on the test-set for class FAMILY OFFENSES is: 0.0369818400488
F1-score on the test-set for class FORGERY/COUNTERFEITING is: 0.0792159971871
F1-score on the test-set for class FRAUD is: 0.145113288014
F1-score on the test-set for class GAMBLING is: 0.125517222984
F1-score on the test-set for class KID

In [491]:
accuracy_score(y_test, tt)

0.11700000000000001

In [492]:
from sklearn.metrics import classification_report
print(classification_report(y_test, tt))

                             precision    recall  f1-score   support

                      ARSON       0.00      0.00      0.00        16
                    ASSAULT       0.21      0.12      0.15       493
                 BAD CHECKS       0.00      0.00      0.00         2
                    BRIBERY       0.00      0.00      0.00         4
                   BURGLARY       0.17      0.06      0.09       243
         DISORDERLY CONDUCT       0.00      0.00      0.00        12
DRIVING UNDER THE INFLUENCE       0.00      0.00      0.00        16
              DRUG/NARCOTIC       0.25      0.27      0.26       155
                DRUNKENNESS       0.00      0.00      0.00        17
               EMBEZZLEMENT       0.50      0.11      0.18         9
            FAMILY OFFENSES       0.00      0.00      0.00         5
     FORGERY/COUNTERFEITING       0.11      0.14      0.12        22
                      FRAUD       0.14      0.14      0.14       113
                   GAMBLING      